In [2]:
import warnings
import json
import os
import re
import time
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import js2py

warnings.filterwarnings("ignore")
print("d2loadout update started")

d2loadout update started


In [3]:
# Configure Selenium WebDriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(f"user-agent={UserAgent().random}")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("disable-infobars")
chrome_options.add_experimental_option(
    "excludeSwitches", ["enable-logging", "enable-automation"]
)
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_experimental_option("detach", True)
chrome_options.page_load_strategy = "eager"
os.environ["PATH"] += r"C:\Program Files (x86)\Chromedriver.exe"
driver = webdriver.Chrome(options=chrome_options)

In [4]:
def get_build(driver):
    def get_titles(elements, attr="title"):
        return [elem.get_attribute(attr) for elem in elements]

    rows = driver.find_element(
        By.CSS_SELECTOR,
        ".grid.grid-cols-3.p-4.gap-1.rounded-md.bg-d-gray-5.border.border-solid.border-d-gray-8",
    ).find_elements(By.XPATH, "./child::*")
    build_talents = [
        int(re.findall(r"(\d+)\s(rt|lt)", talent.get_attribute("class"))[0][1] == "rt")
        for talent in rows[0].find_elements(By.CSS_SELECTOR, ".abs.svelte-yv4bey")
    ]
    build_abilities = get_titles(
        rows[0]
        .find_element(By.CSS_SELECTOR, ".flex.flex-wrap.gap-1.items-center")
        .find_elements(By.XPATH, "./child::img")
    )
    build_starting = get_titles(
        rows[1].find_elements(
            By.CSS_SELECTOR,
            ".flex.flex-col.gap-2.bg-d2pt-gray-1.p-2.rounded-md.border-one",
        ),
        attr="title",
    )
    build_common_late = get_titles(
        rows[2].find_elements(
            By.CSS_SELECTOR,
            ".text-xs.text-white.font-medium.text-right.rounded-md.text-shadow",
        )
    )
    rows_2 = driver.find_elements(
        By.CSS_SELECTOR,
        ".grid.grid-cols-3.gap-1.rounded-md.bg-d-gray-5.border.border-solid.border-d-gray-8",
    )[1].find_elements(By.XPATH, "./child::*")
    build_early = get_titles(
        rows_2[0].find_elements(
            By.CSS_SELECTOR,
            ".flex.flex-col.gap-2.bg-d2pt-gray-1.p-2.rounded-md.border-one",
        ),
        attr="title",
    )
    build_mid = get_titles(
        rows_2[1].find_elements(
            By.CSS_SELECTOR,
            ".flex.flex-col.gap-2.bg-d2pt-gray-1.p-2.rounded-md.border-one",
        ),
        attr="title",
    )
    build_late = get_titles(
        rows_2[2].find_elements(
            By.CSS_SELECTOR,
            ".flex.flex-col.gap-2.bg-d2pt-gray-1.p-2.rounded-md.border-one",
        ),
        attr="title",
    )
    build_core = get_titles(
        driver.find_element(
            By.CSS_SELECTOR,
            ".flex.mb-2.flex-col.p-4.gap-2.rounded-md.bg-d-gray-5.border.border-solid.border-d-gray-8.overflow-x-auto",
        ).find_elements(
            By.CSS_SELECTOR,
            ".flex.gap-1.flex-col.bg-d2pt-gray-1.p-1.rounded-md.border-one.text-xs.text-white.font-medium",
        ),
        attr="title",
    )

    return [
        build_talents,
        build_abilities,
        build_starting,
        build_common_late,
        build_early,
        build_mid,
        build_late,
        build_core,
    ]

In [5]:
# Fetch meta data
link_meta = "https://dota2protracker.com/meta"
driver.implicitly_wait(3)
driver.get(link_meta)
time.sleep(1)
category_name_elements = driver.find_elements(By.XPATH, "//a[@href]")
links = sorted(
    list(
        set(
            [
                elem.get_attribute("href")
                for elem in category_name_elements
                if "/hero/" in elem.get_attribute("href")
            ]
        )
    )
)

In [6]:
def get_d2pt_page_table(driver):
    time.sleep(0.2)
    category_name_elements = driver.find_elements(By.XPATH, "//*[@data-wr]")
    div_inner_strs = []
    for elem in category_name_elements:
        div_inner_str = (
            elem.get_attribute("outerHTML").split(">")[0].replace("data-", "")
        )
        div_items = re.findall(
            r'[-\w]+="[\w\d\s.,\/#!$%\^&\*;:{}=\-_`~()\'\\/\[\]]+"', div_inner_str
        )
        temp_dict = {
            item.split("=")[0]: item.split("=")[1].replace('"', "")
            for item in div_items
        }
        div_inner_strs.append(temp_dict)

    df_heroes_table = (
        pd.DataFrame(data=div_inner_strs).apply(pd.to_numeric, errors="ignore").round(1)
    )
    return df_heroes_table

In [7]:
# Collecting meta data for different positions
positions = ["Carry", "Mid", "Off", "Pos 4", "Pos 5"]
df_meta = pd.concat([get_d2pt_page_table(driver) for pos in positions])
df_meta = df_meta.drop(["class"], axis=1)

# Fetch facets data
link_facets = "https://dota2protracker.com/facets"
driver.get(link_facets)
hero_req = requests.get("https://dota2protracker.com/hero/Tiny")
facet_data = re.findall(r"facetData:{.+}", hero_req.text)[0].replace("facetData:", "")

tojs = re.findall(r"const data = \[.+\"data\":{", hero_req.text)[0]
tojs = tojs + re.findall(r"facetData:{.+},buildData", hero_req.text)[0].replace(
    ",buildData", "}}]"
)
js_res = js2py.eval_js(tojs)
d = js_res[2]["data"]["facetData"]
facets_d = {item: [facet["name"].upper() for facet in d[item]["facets"]] for item in d}

In [8]:
def get_d2pt_page_table_facets(driver):
    time.sleep(0.2)
    category_name_elements = driver.find_elements(
        By.CSS_SELECTOR, ".flex.bg-d2pt-gray-3.gap-1"
    )
    category_name_elements += driver.find_elements(
        By.CSS_SELECTOR, ".flex.bg-d2pt-gray-4.gap-1"
    )
    hero_rows = [item.text.split("\n") for item in category_name_elements]
    hero_columns = [
        item
        for item in driver.find_element(
            By.CSS_SELECTOR, ".flex.gap-1.font-medium.text-sm.mb-1"
        ).text.split("\n")
        if item != "Trend"
    ]

    df_heroes_table = (
        pd.DataFrame(data=hero_rows, columns=hero_columns)
        .apply(pd.to_numeric, errors="ignore")
        .round(1)
    )
    return df_heroes_table

In [9]:
# Collecting facets data for different positions
df_facets = pd.concat([get_d2pt_page_table_facets(driver) for pos in positions])
df_facets = df_facets.rename(
    {"Hero": "hero", "Facet": "facet", "Matches": "matches", "Win Rate": "wr"}, axis=1
)[["hero", "facet", "matches", "wr", "pos"]]
df_facets["wr"] = df_facets["wr"].apply(lambda x: x.replace("%", ""))
df_facets = df_facets.apply(pd.to_numeric, errors="ignore")

KeyError: "['pos'] not in index"

In [ ]:
# Results
print("creating in-game loadouts")
req = requests.get("https://dota2protracker.com/_get/search").json()
hero_names = [item["displayName"] for item in req["heroes"]]
hero_ids = [item["hero_id"] for item in req["heroes"]]
hero_id = dict(zip(hero_names, hero_ids))
df_meta["hero_id"] = df_meta["hero"].apply(lambda x: hero_id[x])
df_facets["hero_id"] = df_facets["hero"].apply(lambda x: hero_id[x])
df_facets["facet_id"] = df_facets.apply(
    lambda x: facets_d[f"{x.hero_id}"].index(x.facet), axis=1
)

In [ ]:
def make_config(get_df_func, df, name):
    margin = 20
    width = 585 - margin
    height = 189 - margin
    not_included = [
        id
        for id in hero_ids
        if id
        not in [j for sub in [get_df_func(df, i) for i in [1, 2, 3, 4, 5]] for j in sub]
    ]

    return {
        "config_name": name,
        "categories": [
            {
                "category_name": pos,
                "x_position": 0,
                "y_position": (height + margin) * i,
                "width": width,
                "height": height,
                "hero_ids": get_df_func(df, i + 1),
            }
            for i, pos in enumerate(
                ["Carry", "Mid", "Offlane", "Semi-Support", "Full-Support"]
            )
        ]
        + [
            {
                "category_name": "Not-Included",
                "x_position": width + margin,
                "y_position": (height + margin) * 2,
                "width": width,
                "height": height,
                "hero_ids": not_included,
            }
        ],
        "slots": [
            {
                "category_name": pos,
                "x_position": 0,
                "y_position": (height + margin) * i,
                "width": width,
                "height": height,
                "hero_ids": get_df_func(df, i + 1),
            }
            for i, pos in enumerate(
                ["Carry", "Mid", "Offlane", "Semi-Support", "Full-Support"]
            )
        ]
        + [
            {
                "category_name": "Not-Included",
                "x_position": width + margin,
                "y_position": (height + margin) * 2,
                "width": width,
                "height": height,
                "hero_ids": not_included,
            }
        ],
    }
    
    
def make_configs(get_df_func, df, names):
    for name in names:
        config = make_config(get_df_func, df, name)
        with open(f"{name}.json", "w", encoding="utf-8") as f:
            json.dump(config, f, ensure_ascii=False, indent=4)

In [ ]:
get_df_func = lambda df, pos: list(df.loc[df.pos == pos]["hero_id"].values)
make_configs(get_df_func, df_meta, ["meta", "meta_testing"])
get_df_func = lambda df, pos: list(
    df.loc[(df.pos == pos) & (df.facet_id == 0)]["hero_id"].values
)
make_configs(get_df_func, df_facets, ["pro", "pro_testing"])
driver.quit()